In [1]:
import logging
import os.path
import time
from collections import OrderedDict
import sys
import mne
import numpy as np
from scipy.io import loadmat


from braindecode.datasets.bcic_iv_2a import BCICompetition4Set2A
from braindecode.experiments.monitors import LossMonitor, MisclassMonitor, \
    RuntimeMonitor
from braindecode.experiments.stopcriteria import MaxEpochs, NoDecrease, Or
from braindecode.datautil.iterators import BalancedBatchSizeIterator
from braindecode.datautil.splitters import split_into_two_sets
from braindecode.mne_ext.signalproc import mne_apply,resample_cnt
from braindecode.datautil.signalproc import (bandpass_cnt,
                                             exponential_running_standardize)
from braindecode.datautil.trial_segment import create_signal_target_from_raw_mne

In [2]:
import numpy as np
import mne
from scipy.io import loadmat


class BCICompetition4Set2A(object):
    def __init__(self, filename, load_sensor_names=None, labels_filename=None):
        assert load_sensor_names is None
        self.__dict__.update(locals())
        del self.self

    def load(self):
        cnt = self.extract_data()
        events, artifact_trial_mask = self.extract_events(cnt)
        cnt.info["events"] = events
        cnt.info["artifact_trial_mask"] = artifact_trial_mask
        return cnt

    def extract_data(self):
        raw_gdf = mne.io.read_raw_gdf(self.filename, stim_channel="auto")
        raw_gdf.load_data()
        # correct nan values

        data = raw_gdf.get_data()

        for i_chan in range(data.shape[0]):
            # first set to nan, than replace nans by nanmean.
            this_chan = data[i_chan]
            data[i_chan] = np.where(
                this_chan == np.min(this_chan), np.nan, this_chan
            )
            mask = np.isnan(data[i_chan])
            chan_mean = np.nanmean(data[i_chan])
            data[i_chan, mask] = chan_mean

        gdf_events = mne.events_from_annotations(raw_gdf)
        raw_gdf = mne.io.RawArray(data, raw_gdf.info, verbose="WARNING")
        # remember gdf events
        raw_gdf.info["gdf_events"] = gdf_events
        return raw_gdf

    def extract_events(self, raw_gdf):
        # all events
        events, name_to_code = raw_gdf.info["gdf_events"]
        print(name_to_code)
        train_set = True
        if "class1, Left hand - cue onset (BCI experiment)" in name_to_code:
            train_set = True
        else:
            train_set = False
            assert (
                "cue unknown/undefined (used for BCI competition) "
                in name_to_code
            )

        if train_set:
            trial_codes = [4, 5, 6, 7]  # the 4 classes
        else:
            trial_codes = [4]  # "unknown" class

        trial_mask = [ev_code in trial_codes for ev_code in events[:, 2]]
        trial_events = events[trial_mask]
        assert len(trial_events) == 288, "Got {:d} markers".format(
            len(trial_events)
        )
        trial_events[:, 2] = trial_events[:, 2] - 3
        # possibly overwrite with markers from labels file
        if self.labels_filename is not None:
            classes = loadmat(self.labels_filename)["classlabel"].squeeze()
            if train_set:
                np.testing.assert_array_equal(trial_events[:, 2], classes)
            trial_events[:, 2] = classes
        unique_classes = np.unique(trial_events[:, 2])
        assert np.array_equal(
            [1, 2, 3, 4], unique_classes
        ), "Expect 1,2,3,4 as class labels, got {:s}".format(
            str(unique_classes)
        )

        # now also create 0-1 vector for rejected trials
        trial_start_events = events[events[:, 2] == 2]
        assert len(trial_start_events) == len(trial_events)
        artifact_trial_mask = np.zeros(len(trial_events), dtype=np.uint8)
        artifact_events = events[events[:, 2] == 1]

        for artifact_time in artifact_events[:, 0]:
            i_trial = trial_start_events[:, 0].tolist().index(artifact_time)
            artifact_trial_mask[i_trial] = 1

        return trial_events, artifact_trial_mask

In [43]:
filename = "bci_2a/A01T.gdf"
labels_filename = "bci_2a/A01T.mat"

In [44]:
raw_gdf = mne.io.read_raw_gdf(filename, stim_channel="auto")
raw_gdf.load_data()
# correct nan values

data = raw_gdf.get_data()

Extracting EDF parameters from /Users/noahcarniglia/189project/bci_2a/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/Users/noahcarniglia/anaconda3/lib/python3.7/site-packages/mne/io/edf/edf.py:996: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, np.uint8).tolist()[0]
<ipython-input-44-2b7a3a22d3a5>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw_gdf = mne.io.read_raw_gdf(filename, stim_channel="auto")


In [45]:
data.shape

(25, 672528)

In [46]:
raw_gdf

<RawGDF  |  A01T.gdf, n_channels x n_times : 25 x 672528 (2690.1 sec), ~128.3 MB, data loaded>

In [47]:
for i_chan in range(data.shape[0]):
    # first set to nan, than replace nans by nanmean.
    this_chan = data[i_chan]
    data[i_chan] = np.where(
        this_chan == np.min(this_chan), np.nan, this_chan
    )
    mask = np.isnan(data[i_chan])
    chan_mean = np.nanmean(data[i_chan])
    data[i_chan, mask] = chan_mean

In [48]:
gdf_events = mne.events_from_annotations(raw_gdf)
raw_gdf = mne.io.RawArray(data, raw_gdf.info, verbose="WARNING")

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [49]:
gdf_events

(array([[     0,      0,      5],
        [     0,      0,      3],
        [ 29683,      0,      5],
        ...,
        [670550,      0,      6],
        [670550,      0,      1],
        [671050,      0,      7]]),
 {'1023': 1,
  '1072': 2,
  '276': 3,
  '277': 4,
  '32766': 5,
  '768': 6,
  '769': 7,
  '770': 8,
  '771': 9,
  '772': 10})

In [50]:
raw_gdf.info["gdf_events"] = gdf_events

In [51]:
raw_gdf

<RawArray  |  None, n_channels x n_times : 25 x 672528 (2690.1 sec), ~128.3 MB, data loaded>

In [52]:
events, name_to_code = raw_gdf.info["gdf_events"]

In [53]:
name_to_code

{'1023': 1,
 '1072': 2,
 '276': 3,
 '277': 4,
 '32766': 5,
 '768': 6,
 '769': 7,
 '770': 8,
 '771': 9,
 '772': 10}

In [54]:
train_set = True

In [55]:
if train_set:
    trial_codes = [7,8,9,10]  # the 4 classes
else:
    trial_codes = [4]  # "unknown" class

trial_mask = [ev_code in trial_codes for ev_code in events[:, 2]]
trial_events = events[trial_mask]
assert len(trial_events) == 288, "Got {:d} markers".format(
    len(trial_events)
)

In [56]:
events

array([[     0,      0,      5],
       [     0,      0,      3],
       [ 29683,      0,      5],
       ...,
       [670550,      0,      6],
       [670550,      0,      1],
       [671050,      0,      7]])

In [57]:
trial_mask

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 Fal

In [60]:
trial_events

array([[ 92368,      0,      7],
       [ 94371,      0,      6],
       [ 96289,      0,      5],
       [ 98241,      0,      4],
       [100249,      0,      4],
       [102360,      0,      5],
       [104277,      0,      6],
       [106327,      0,      7],
       [108258,      0,      5],
       [110256,      0,      6],
       [112162,      0,      4],
       [114057,      0,      4],
       [116029,      0,      4],
       [117973,      0,      7],
       [119940,      0,      5],
       [122060,      0,      5],
       [124068,      0,      4],
       [126134,      0,      4],
       [128170,      0,      6],
       [130236,      0,      4],
       [132306,      0,      5],
       [134386,      0,      7],
       [136298,      0,      7],
       [138329,      0,      6],
       [140282,      0,      4],
       [142243,      0,      7],
       [144347,      0,      7],
       [146351,      0,      5],
       [148326,      0,      7],
       [150352,      0,      7],
       [15

In [61]:
trial_events[:, 2] = trial_events[:, 2] - 3
# possibly overwrite with markers from labels file
if labels_filename is not None:
    classes = loadmat(labels_filename)["classlabel"].squeeze()
    if train_set:
        np.testing.assert_array_equal(trial_events[:, 2], classes)
    trial_events[:, 2] = classes
unique_classes = np.unique(trial_events[:, 2])

In [66]:
# now also create 0-1 vector for rejected trials
trial_start_events = events[events[:, 2] == 6]
assert len(trial_start_events) == len(trial_events)
artifact_trial_mask = np.zeros(len(trial_events), dtype=np.uint8)
artifact_events = events[events[:, 2] == 1]

for artifact_time in artifact_events[:, 0]:
    i_trial = trial_start_events[:, 0].tolist().index(artifact_time)
    artifact_trial_mask[i_trial] = 1

In [67]:
events

array([[     0,      0,      5],
       [     0,      0,      3],
       [ 29683,      0,      5],
       ...,
       [670550,      0,      6],
       [670550,      0,      1],
       [671050,      0,      7]])

In [68]:
trial_events

array([[ 92368,      0,      4],
       [ 94371,      0,      3],
       [ 96289,      0,      2],
       [ 98241,      0,      1],
       [100249,      0,      1],
       [102360,      0,      2],
       [104277,      0,      3],
       [106327,      0,      4],
       [108258,      0,      2],
       [110256,      0,      3],
       [112162,      0,      1],
       [114057,      0,      1],
       [116029,      0,      1],
       [117973,      0,      4],
       [119940,      0,      2],
       [122060,      0,      2],
       [124068,      0,      1],
       [126134,      0,      1],
       [128170,      0,      3],
       [130236,      0,      1],
       [132306,      0,      2],
       [134386,      0,      4],
       [136298,      0,      4],
       [138329,      0,      3],
       [140282,      0,      1],
       [142243,      0,      4],
       [144347,      0,      4],
       [146351,      0,      2],
       [148326,      0,      4],
       [150352,      0,      4],
       [15

In [16]:
subject_id = 1
data_folder = 'bci_2a/'

train_filename = "A{:02d}T.gdf".format(subject_id)
test_filename = "A{:02d}E.gdf".format(subject_id)
train_filepath = os.path.join(data_folder, train_filename)
test_filepath = os.path.join(data_folder, test_filename)
train_label_filepath = train_filepath.replace(".gdf", ".mat")
test_label_filepath = test_filepath.replace(".gdf", ".mat")

print(train_filepath)


train_loader = BCICompetition4Set2A(train_filepath, labels_filename=train_label_filename)
test_loader = BCICompetition4Set2A(test_filepath, labels_filename=test_label_filename)

train_cnt = train_loader.load()
test_cnt = test_loader.load()